# Data Concatenation and Cleaning

This notebook concatenates three CSV files (metasploitable-2.csv, Normal_data.csv, OVS.csv) from the data folder, performs basic data cleaning, and analyzes the labels in the concatenated dataset. The output is saved as concatenated_data.csv in the output folder.

In [11]:
# Import required libraries
import pandas as pd
import numpy as np
import os

# Define paths
data_dir = '../data/'
output_dir = '../output/'
output_file = os.path.join(output_dir, 'concatenated_data.csv')

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# List of CSV files
csv_files = ['metasploitable-2.csv', 'Normal_data.csv', 'OVS.csv']

# Initialize an empty list to store DataFrames
dfs = []

# Read and concatenate CSV files
for file in csv_files:
    file_path = os.path.join(data_dir, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if 'Label' in df.columns:
            df['Label'] = df['Label'].str.strip()  # Remove leading/trailing whitespace
        dfs.append(df)
        print(f'Loaded {file} with shape {df.shape}')
    else:
        print(f'File {file} not found')

# Concatenate all DataFrames
if dfs:
    concatenated_df = pd.concat(dfs, ignore_index=True)
    print(f'\nConcatenated DataFrame shape: {concatenated_df.shape}')
else:
    print('No DataFrames to concatenate')
    concatenated_df = pd.DataFrame()

Loaded metasploitable-2.csv with shape (136743, 84)
Loaded Normal_data.csv with shape (68424, 84)
Loaded OVS.csv with shape (138722, 84)

Concatenated DataFrame shape: (343889, 84)


In [8]:
# Basic data exploration
if not concatenated_df.empty:
    print('\nFirst 5 rows of concatenated DataFrame:')
    display(concatenated_df.head())
    
    print('\nDataFrame Info:')
    concatenated_df.info()
    
    print('\nMissing Values:')
    print(concatenated_df.isnull().sum())
    
    print('\nDuplicate Rows:')
    print(concatenated_df.duplicated().sum())


First 5 rows of concatenated DataFrame:


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.3.130-200.175.2.130-38694-4444-6,192.168.3.130,38694,200.175.2.130,4444,6,10/1/2020 5:02,269709,4,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,U2R
1,192.168.3.130-200.175.2.130-38693-4444-6,192.168.3.130,38693,200.175.2.130,4444,6,10/1/2020 5:02,268599,2,3,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,U2R
2,192.168.3.130-200.175.2.130-3632-33747-6,200.175.2.130,33747,192.168.3.130,3632,6,10/1/2020 5:02,22194,5,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,U2R
3,192.168.3.130-200.175.2.130-8180-38745-6,200.175.2.130,38745,192.168.3.130,8180,6,10/1/2020 1:39,9556,4,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA
4,192.168.3.130-200.175.2.130-8180-37217-6,200.175.2.130,37217,192.168.3.130,8180,6,10/1/2020 1:39,8782,4,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343889 entries, 0 to 343888
Data columns (total 84 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow ID            343889 non-null  object 
 1   Src IP             343889 non-null  object 
 2   Src Port           343889 non-null  int64  
 3   Dst IP             343889 non-null  object 
 4   Dst Port           343889 non-null  int64  
 5   Protocol           343889 non-null  int64  
 6   Timestamp          343889 non-null  object 
 7   Flow Duration      343889 non-null  int64  
 8   Tot Fwd Pkts       343889 non-null  int64  
 9   Tot Bwd Pkts       343889 non-null  int64  
 10  TotLen Fwd Pkts    343889 non-null  float64
 11  TotLen Bwd Pkts    343889 non-null  float64
 12  Fwd Pkt Len Max    343889 non-null  int64  
 13  Fwd Pkt Len Min    343889 non-null  int64  
 14  Fwd Pkt Len Mean   343889 non-null  float64
 15  Fwd Pkt Len Std    343889 non-null

In [5]:
# Data cleaning
if not concatenated_df.empty:
    # Remove duplicate rows
    concatenated_df = concatenated_df.drop_duplicates()
    print(f'Shape after removing duplicates: {concatenated_df.shape}')
    
    # Handle missing values (example: fill numeric with mean, categorical with mode)
    for column in concatenated_df.columns:
        if concatenated_df[column].dtype in ['int64', 'float64']:
            concatenated_df[column].fillna(concatenated_df[column].mean(), inplace=True)
        else:
            concatenated_df[column].fillna(concatenated_df[column].mode()[0], inplace=True)
    
    print('\nMissing Values After Cleaning:')
    print(concatenated_df.isnull().sum())
    
    # Save the cleaned DataFrame
    concatenated_df.to_csv(output_file, index=False)
    print(f'\nCleaned data saved to {output_file}')

Shape after removing duplicates: (205166, 84)

Missing Values After Cleaning:
Flow ID      0
Src IP       0
Src Port     0
Dst IP       0
Dst Port     0
            ..
Idle Mean    0
Idle Std     0
Idle Max     0
Idle Min     0
Label        0
Length: 84, dtype: int64


/var/folders/np/ds9v8zlx33j30nx7p_2wlx_80000gn/T/ipykernel_4669/3126887602.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  concatenated_df[column].fillna(concatenated_df[column].mode()[0], inplace=True)
/var/folders/np/ds9v8zlx33j30nx7p_2wlx_80000gn/T/ipykernel_4669/3126887602.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on w


Cleaned data saved to ../output/concatenated_data.csv


## Next Steps

The concatenated and cleaned data is saved as `concatenated_data.csv` in the output folder. Proceed with applying federated learning concepts.

In [12]:
# Label analysis
if not concatenated_df.empty:
    if 'Label' in concatenated_df.columns:
        print(f'\nShape of combined data: {concatenated_df.shape}')
        unique_labels = concatenated_df['Label'].unique()
        print('\nUnique labels (attack types):', unique_labels.tolist())
        label_counts = concatenated_df['Label'].value_counts()
        print('\nCount of each label:')
        print(label_counts)
    else:
        print("\nError: 'Label' column not found in the DataFrame. Please check column names.")
        print('Available columns:', concatenated_df.columns.tolist())


Shape of combined data: (343889, 84)

Unique labels (attack types): ['U2R', 'BFA', 'DDoS', 'DoS', 'Probe', 'Normal', 'Web-Attack', 'BOTNET']

Count of each label:
Label
DDoS          121942
Probe          98129
Normal         68424
DoS            53616
BFA             1405
Web-Attack       192
BOTNET           164
U2R               17
Name: count, dtype: int64


In [10]:
# Verify raw CSV files
print('\nInspecting raw CSV files:')
for file in ['metasploitable-2.csv', 'Normal_data.csv', 'OVS.csv']:
    file_path = os.path.join(data_dir, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f'\n{file} shape: {df.shape}')
        if 'Label' in df.columns:
            print(f'Unique labels in {file}:', df['Label'].unique().tolist())
            print(f'Label counts in {file}:\n{df["Label"].value_counts()}')
        else:
            print(f"No 'Label' column in {file}. Available columns:", df.columns.tolist())


Inspecting raw CSV files:

metasploitable-2.csv shape: (136743, 84)
Unique labels in metasploitable-2.csv: ['U2R', 'BFA', 'DDoS', 'DoS', 'Probe']
Label counts in metasploitable-2.csv:
Label
DDoS     73529
Probe    61757
DoS       1145
BFA        295
U2R         17
Name: count, dtype: int64

Normal_data.csv shape: (68424, 84)
Unique labels in Normal_data.csv: ['Normal']
Label counts in Normal_data.csv:
Label
Normal    68424
Name: count, dtype: int64

OVS.csv shape: (138722, 84)
Unique labels in OVS.csv: ['BFA', 'DDoS ', 'DoS', 'Probe', 'Web-Attack', 'BOTNET']
Label counts in OVS.csv:
Label
DoS           52471
DDoS          48413
Probe         36372
BFA            1110
Web-Attack      192
BOTNET          164
Name: count, dtype: int64


## Next Steps

The concatenated and cleaned data is saved as `concatenated_data.csv` in the output folder. The label analysis above shows the distribution of attack types for metasploitable-2.csv, Normal_data.csv, and OVS.csv. If the output matches expectations, proceed with federated learning. If labels or row counts differ from expected, review the raw CSV inspection above.

## Next Steps

The concatenated and cleaned data is saved as `concatenated_data.csv` in the output folder. The label analysis above shows the distribution of attack types for metasploitable-2.csv, Normal_data.csv, and OVS.csv, with 'DDoS' and 'DDoS ' combined into a single label. The data is then clubbed into three files (ddos_data.csv, dos_data.csv, probe_data.csv) in the output_clubbed folder, each including Normal and specific attack types. Proceed with federated learning using these files.

In [13]:
# Club data into three categories
if not concatenated_df.empty and 'Label' in concatenated_df.columns:
    # Define output directory for clubbed data
    clubbed_output_dir = '../output_clubbed/'
    os.makedirs(clubbed_output_dir, exist_ok=True)
    
    # Define label groups
    ddos_labels = ['DDoS', 'Normal', 'BFA']
    dos_labels = ['Probe', 'Normal', 'Web-Attack']
    probe_labels = ['Normal', 'DoS', 'BOTNET']
    
    # Create and save clubbed datasets
    ddos_data = concatenated_df[concatenated_df['Label'].isin(ddos_labels)]
    ddos_data.to_csv(os.path.join(clubbed_output_dir, 'ddos_data.csv'), index=False)
    print(f'Saved ddos_data.csv with shape {ddos_data.shape} and labels {ddos_data["Label"].unique().tolist()}')
    
    dos_data = concatenated_df[concatenated_df['Label'].isin(dos_labels)]
    dos_data.to_csv(os.path.join(clubbed_output_dir, 'dos_data.csv'), index=False)
    print(f'Saved dos_data.csv with shape {dos_data.shape} and labels {dos_data["Label"].unique().tolist()}')
    
    probe_data = concatenated_df[concatenated_df['Label'].isin(probe_labels)]
    probe_data.to_csv(os.path.join(clubbed_output_dir, 'probe_data.csv'), index=False)
    print(f'Saved probe_data.csv with shape {probe_data.shape} and labels {probe_data["Label"].unique().tolist()}')
else:
    print('Error: concatenated_df is empty or Label column not found')

Saved ddos_data.csv with shape (191771, 84) and labels ['BFA', 'DDoS', 'Normal']
Saved dos_data.csv with shape (166745, 84) and labels ['Probe', 'Normal', 'Web-Attack']
Saved probe_data.csv with shape (122204, 84) and labels ['DoS', 'Normal', 'BOTNET']


In [14]:
for file in ['ddos_data.csv', 'dos_data.csv', 'probe_data.csv']:
    df = pd.read_csv(os.path.join(clubbed_output_dir, file))
    print(f'\n{file} shape: {df.shape}')
    print(f'Labels: {df["Label"].unique().tolist()}')


ddos_data.csv shape: (191771, 84)
Labels: ['BFA', 'DDoS', 'Normal']

dos_data.csv shape: (166745, 84)
Labels: ['Probe', 'Normal', 'Web-Attack']

probe_data.csv shape: (122204, 84)
Labels: ['DoS', 'Normal', 'BOTNET']
